<a href="https://colab.research.google.com/github/iamshirdi/Datascience/blob/master/python/sqlite_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h4>Counting Email  Organisations in a Database

In [46]:
!wget -O mbox-short.txt "http://www.py4e.com/code3/mbox.txt"
import sqlite3


conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'mbox-short.txt'
fh = open(fname)

for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    p2=pieces[1].split('@')
    org = p2[1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,)) #sql injection prevention tuple
    
    row = cur.fetchone() #note cursor is prepared with email from above statement and fetch checks it by given below if statment
    #The  fetchone() method returns the next row of a query result set or None in case there is no row left
    
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
        
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()
# Because the sample code is using an UPDATE statement and
#committing the results to the database as each record is read in the loop, it might take as long as a few minutes to process all the data.
#The commit insists on completely writing all the data to disk every time it is called. 

--2019-06-18 11:59:20--  http://www.py4e.com/code3/mbox.txt
Resolving www.py4e.com (www.py4e.com)... 104.27.158.166, 104.27.159.166, 2606:4700:30::681b:9ea6, ...
Connecting to www.py4e.com (www.py4e.com)|104.27.158.166|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.py4e.com/code3/mbox.txt [following]
--2019-06-18 11:59:20--  https://www.py4e.com/code3/mbox.txt
Connecting to www.py4e.com (www.py4e.com)|104.27.158.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘mbox-short.txt’

mbox-short.txt          [   <=>              ]   6.38M  9.00MB/s    in 0.7s    

2019-06-18 11:59:22 (9.00 MB/s) - ‘mbox-short.txt’ saved [6687002]

Enter file name: 
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [47]:
! curl --upload-file ./emaildb.sqlite https://transfer.sh/emaildb.sqlite

https://transfer.sh/14lXMQ/emaildb.sqlite

<h4>SQLite browser to make a database, insert some data and then run query.

In [30]:
import sqlite3

conn = sqlite3.connect('sql1db.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Ages')

In [31]:
cur.execute('''CREATE TABLE Ages ( 
  name VARCHAR(128), 
  age INTEGER
)''')

In [32]:
db1="""DELETE FROM Ages;
INSERT INTO Ages (name, age) VALUES ('Satveer', 32);
INSERT INTO Ages (name, age) VALUES ('Farrah', 24);
INSERT INTO Ages (name, age) VALUES ('Sayuri', 29);
INSERT INTO Ages (name, age) VALUES ('Ehsan', 34);"""
cur.executescript(db1) #multiline command

In [33]:
print(cur.execute('''SELECT hex(name || age) AS X FROM Ages ORDER BY X'''))
sqlobj=cur.execute('SELECT hex(name || age) AS X FROM Ages ORDER BY X')

In [34]:
# for row in sqlobj:
#     print(str(row[0]))
for row in cur: #cursor.exectute in which cursor also has it
  print(row)

('456873616E3334',)
('4661727261683234',)
('536174766565723332',)
('5361797572693239',)


In [0]:
conn.commit() #commits to database
cur.close()